In [1]:
using IRQIV

# Global camera parameters
fl = 1214.2857142857144 # In pixels - from Seth
include(raw"/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl/examples/CameraCalibrationHelpers.jl")
Camera = IRQIV.FLIR_SC8303_17mm
ι = IRQIV.IntrinsicCalibrationParameters(Camera)

IntrinsicCalibrationParameters(672.5, 392.5, 1.0086485610865155, 1.0, 1.4e-5, [1.0, 3.0, 5.0], [0.020522291524825344, -3.156396095419575e-8, -8.859823756718999e-14])

In [2]:
# 
import Dates 

ats_year = 2022

first_filenum = 3486
last_filenum = 3506

first_filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string.(first_filenum) * ".ats")
first_images = LoadATSImageSequence(first_filename, Inf, ats_year)

last_filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string.(last_filenum) * ".ats")
last_images = LoadATSImageSequence(last_filename, Inf, ats_year)

# Verify framerate is roughly constant from first to last file
dump(first_images.Metadata.Estimatedfps)
dump(last_images.Metadata.Estimatedfps)

Float64 8.886111111111111
Float64 8.861520392023246


In [3]:
file_range = first_filenum:last_filenum

ATS_start_times = fill(Dates.DateTime(0), length(first_filenum:last_filenum))
ATS_framerates = zeros(length(first_filenum:last_filenum))


for (i, file) in enumerate(first_filenum:last_filenum)
    filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string.(file) * ".ats")
    first_frames = LoadATSImageSequence(filename, 5, ats_year)

    ATS_start_times[i] = ats_ts(first_frames)[1]
    ATS_framerates[i] = first_frames.Metadata.Estimatedfps

    dump(ATS_start_times[i])
    dump(ATS_framerates[i])
end


Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793511563402
Float64 8.98876404494382
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793511959402
Float64 9.00900900900901
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793512355513
Float64 9.00900900900901
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793512752735
Float64 9.00900900900901
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793513149512
Float64 8.98876404494382
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
      value: Int64 63793513545623
Float64 8.98876404494382
Dates.DateTime
  instant: Dates.UTInstant{Dates.Millisecond}
    periods: Dates.Millisecond
  

In [6]:
import Dates
using Statistics  # Ensure to import Statistics for standard deviation calculations

ms_to_s = Dates.Millisecond(1000)

# Define file range
file_range = first_filenum:last_filenum


# Pre-allocate the variance matrix
std_frames = zeros(axes(first_images)[1], axes(first_images)[2], length(file_range))

for (file_ind, file) in enumerate(file_range)
    filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string(file) * ".ats")

    start_frame = 2000

    stop_frame = 30*9 # 30 sec at 9hz

    frames = LoadATSImageSequence(filename, start_frame:start_frame+stop_frame, ats_year)[:,:,:]
    
    std_frames[:, :, file_ind] = dropdims(Statistics.std(frames, dims = 3), dims = 3)

    println(file_ind)
end




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [ ]:
plt.pygui(true) # sets plots to popup instead of inline
import PyPlot as plt

# Loop through each time slice in std_frames and display it
for i in 2:size(std_frames, 3) # Skip first file
    start_time = ATS_start_times[i]
    end_time = start_time + Dates.Second(30)
    start_time = Dates.format(start_time, "HH:MM:SS") 
    end_time = Dates.format(end_time, "HH:MM:SS") 
    plt.figure()  # Create a new figure for each slice
    plt.imshow(std_frames[:, :, i], cmap="viridis", vmin=0, vmax=140)  # Display slice with a color map (e.g., "viridis")
    plt.colorbar(label="Standard Deviation")  # Add a color bar
    plt.title("Pixel-wise standard deviation: $start_time to $end_time")  # Title for each plot
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.show()  # Display the plot
end


In [13]:
import PyPlot as plt
import Dates
using PyCall
@pyimport matplotlib.animation as animation

# Create figure and axis for the animation
fig, ax = plt.subplots()

# Set color limits and colorbar
cax = ax.imshow(std_frames[:, :, 2], cmap="viridis", vmin=0, vmax=140)  # Start from second frame
cbar = fig.colorbar(cax, ax=ax, label="Standard Deviation")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")

# Update function for each frame
function update(i)
    ax.clear()  # Clear the axis to redraw
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    
    # Set the time range for the current frame
    start_time = ATS_start_times[i]
    end_time = start_time + Dates.Second(30)
    start_time_str = Dates.format(start_time, "HH:MM:SS")
    end_time_str = Dates.format(end_time, "HH:MM:SS")
    
    # Update image and title
    cax = ax.imshow(std_frames[:, :, i], cmap="viridis", vmin=0, vmax=140)
    ax.set_title("Pixel-wise standard deviation: $start_time_str to $end_time_str")
    return cax  # Return the image to update in animation
end

# Use FuncAnimation from PyPlot's matplotlib to create the animation, starting from frame 2
ani = plt.matplotlib.animation.FuncAnimation(fig, update, frames=3:size(std_frames, 3), repeat=false)

# Save the animation
ani.save("std_frames_animation.gif", dpi = 400, fps=1.5)  # Adjust fps as needed

plt.show()


MovieWriter ffmpeg unavailable; using Pillow instead.
sys:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [ ]:
import Dates
using Statistics  # Ensure to import Statistics for standard deviation calculations

ms_to_s = Dates.Millisecond(1000)

# Define file range
file_range = first_filenum:last_filenum

first_frame = ats_ts(first_images)[1]
first_minute = round(first_frame, Dates.Minute(1))
last_frame = ats_ts(last_images)[end]
last_minute = round(last_frame, Dates.Minute(1))

ts_range = first_minute:Dates.Minute(5):last_minute

# Pre-allocate the variance matrix
std_frames = zeros(axes(first_images)[1], axes(first_images)[2], length(ts_range))

for (time_ind, minute) in enumerate(ts_range)
    start_time = minute
    stop_time = minute + Dates.Minute(1)

    # Initialize an empty array to store the combined frames
    combined_frames = Array{UInt16, 3}(undef, size(first_images)[1], size(first_images)[2], 0)

    for (file_ind, file) in enumerate(file_range)
        filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string(file) * ".ats")
        
        file_start_time = ATS_start_times[file_ind]
        println(file_start_time)
        framerate = ATS_framerates[file_ind]

        # Determine the end time for the file based on 3200 frames
        file_end_time = file_start_time + Dates.Second(3200 / framerate)
        
        # Check if the current interval overlaps with this file
        if stop_time < file_start_time || start_time > file_end_time
            continue  # Skip to the next file if this interval is outside this file's time range
        end
        
        # Determine the frame indices for the current file
        if start_time >= file_start_time && stop_time <= file_end_time

            # Entire interval within this file
            start_frame = round((start_time - file_start_time) / ms_to_s * framerate)
            stop_frame = round((stop_time - file_start_time) / ms_to_s * framerate)
            
            # Ensure frame indices are within the file's range of 3200 frames
            println("calculating start_frame 1")
            start_frame = max(Int(start_frame), 1)
            println(start_frame)

            println("calculating stop_frame 1")
            stop_frame = min(Int(stop_frame), 3200)
            println(stop_frame)

            # Load frames for the calculated range
            if start_frame <= stop_frame  # Ensure valid frame range
                frame_indices = start_frame:stop_frame
                frames = LoadATSImageSequence(filename, frame_indices, ats_year)[:,:,:] # Load as matrix instead of struct
                combined_frames = cat(combined_frames, frames, dims=3) # Add frames to combined array
                println("Size of combined_frames 1: ", size(combined_frames))
            end

        elseif start_time < file_end_time && stop_time > file_start_time
            # Split interval between this file and the next one
            if file_ind < length(file_range)  # Ensure there's a next file
                # Calculate frames from this file
                println("calculating start_frame 2")
                start_frame = round((start_time - file_start_time) / ms_to_s * framerate)
                println(start_frame)

                println("calculating stop_frame 2")
                stop_frame = round((file_end_time - file_start_time) / ms_to_s * framerate)
                println(stop_frame)
                
                # Ensure frame indices are within the file's range of 3200 frames
                start_frame = max(Int(start_frame), 1)
                stop_frame = min(Int(stop_frame), 3200)

                # Load frames for the first part of the interval
                if start_frame <= stop_frame  # Ensure valid frame range
                    frame_indices = start_frame:stop_frame
                    frames_first_part = LoadATSImageSequence(filename, frame_indices, ats_year)[:,:,:]
                    combined_frames = cat(combined_frames, frames, dims=3)
                    println("Size of combined_frames 2: ", size(combined_frames))
                end

                # Now load frames from the next file
                next_file = file + 1
                next_filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string(next_file) * ".ats")
                
                # Get start time for the next file
                next_file_start_time = ATS_start_times[next_file]
                
                # Calculate frame indices for the second part
                start_frame_next = 1  # First frame of the next file
                println("calculating stop_frame_next")
                stop_frame_next = round((stop_time - next_file_start_time) / ms_to_s * framerate)
                println(stop_frame_next)
                
                # Ensure frame indices are within the file's range of 3200 frames
                stop_frame_next = min(Int(stop_frame_next), 3200)

                # Load frames for the second part of the interval
                if start_frame_next <= stop_frame_next  # Ensure valid frame range
                    frame_indices_next = start_frame_next:stop_frame_next
                    frames_second_part = LoadATSImageSequence(next_filename, frame_indices_next, ats_year)[:,:,:]
                    combined_frames = cat(combined_frames, frames, dims=3)  # Add frames to combined array
                    println("Size of combined_frames 3: ", size(combined_frames))
                end
            end
        end
    end


    # Calculate standard deviation across the combined frames and store in variance matrix
    if !isempty(combined_frames)  # Check if combined_frames is not empty
        std_frames[:, :, time_ind] = dropdims(Statistics.std(combined_frames, dims = 3), dims = 3)
        println("Size of std_frames: ", size(std_frames))
    end
end
